In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta


In [62]:
data = pd.read_csv('../data/05_model_input/filtered_df.csv')
data

,LIB_SEM,CATEG_DIAG,NB_PATIENTS,NB_IS_DEATH,NB_IS_CRITIC
0,2023-01-02,COVID_19,80,5,20
1,2023-01-09,COVID_19,68,3,12
2,2023-01-16,COVID_19,45,4,10
3,2023-01-23,COVID_19,54,4,6
4,2023-01-30,COVID_19,35,2,6
...,...,...,...,...,...
113,2023-03-27,RSV,1,0,1
114,2023-04-03,RSV,2,0,1
115,2023-04-10,RSV,1,0,0
116,2023-04-17,RSV,2,0,0


In [63]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [64]:
# Créez une instance de modèle de régression linéaire
model = LinearRegression()

In [65]:
selected_disease = "COVID_19"
selected_parameter = "NB_PATIENTS"



In [66]:
import pandas as pd

# Assurez-vous que les données sont triées par semaine
selected_data = data[data['CATEG_DIAG'] == selected_disease][["LIB_SEM"]]
selected_data["LIB_SEM"] = pd.to_datetime(selected_data["LIB_SEM"])  # Conversion en datetime
selected_data = selected_data.sort_values(by="LIB_SEM")

# Calculez la différence en semaines entre chaque date et la première date
selected_data["WeekDiff"] = (selected_data["LIB_SEM"] - selected_data["LIB_SEM"].min()).dt.days // 7

In [80]:
weeks = len(selected_data["WeekDiff"] ) // 7
weeks * 7 - 14

27

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
from statsmodels.stats.multitest import multipletests

def prediction(rel_data, CL=0.95, pred_hor, cov_mat, simple=True):
    weeks = len(rel_data) // 7
    pred_mat = []
    i = 0
    
    while i <= weeks * 7 - 14:
        days = np.arange(len(rel_data))
        curr_days = np.arange(i, i + 14)
        curr_window = rel_data[curr_days]
        pred_days = np.arange(i + 14, i + 14 + pred_hor)
        
        if simple:
            new_dat = pd.DataFrame({'curr_days': pred_days})
            linreg = linregress(curr_days, np.log(curr_window))
            slope, intercept, r_value, p_value, std_err = linreg
            pred = np.exp(intercept + slope * new_dat['curr_days'])
            conf_int = np.array([np.nan, np.nan])
            pred_mat.append([pred, *conf_int, np.nan])
        
        else:
            cov_dfd = pd.DataFrame({'days': days, **cov_mat})
            new_dat = cov_dfd.iloc[pred_days]
            curr_dat = cov_dfd.iloc[curr_days]
            linreg = linregress(curr_dat, np.log(curr_window))
            slope, intercept, r_value, p_value, std_err = linreg
            pred = np.exp(intercept + slope * new_dat)
            linreg_0 = linregress(curr_dat['days'], np.log(curr_window))
            p_val = var_test(linreg, linreg_0)
            conf_int = np.array([np.nan, np.nan])
            pred_mat.append([pred, *conf_int, p_val])
        
        i += pred_hor
    
    pred_mat = np.array(pred_mat)[1:]
    pred_mat = pred_mat[:, ~np.all(np.isnan(pred_mat), axis=0)]
    
    if pred_mat.shape[1] == 4:
        pred_mat[:, 3] = multipletests(pred_mat[:, 3], method='holm')[1]
    
    return pred_mat

In [67]:
y = data[data['CATEG_DIAG'] == selected_disease]["NB_PATIENTS"]
X = selected_data["WeekDiff"].values.reshape(-1, 1)


In [68]:
# Ajustez le modèle aux données
model.fit(X, y)

LinearRegression()

In [69]:
import numpy as np

time_series = [datetime.strptime(date, "%Y-%m-%d") for date in data['LIB_SEM']]
# Trouvez la dernière semaine dans les données
derniere_semaine = max(time_series)
# Créez une nouvelle DataFrame pour les deux semaines suivantes
nouvelles_semaines = pd.DataFrame({'Week': [derniere_semaine + timedelta(weeks=1), derniere_semaine + timedelta(weeks=2)]})

nouvelles_semaines["Week"] = nouvelles_semaines["Week"].values.astype(np.float64)

# Prédictions pour les deux nouvelles semaines
predictions = model.predict(nouvelles_semaines)

c:\ProgramData\Anaconda3\envs\kedro-environment\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [70]:
# Coefficients de régression
coefficients = model.coef_
intercept = model.intercept_

In [71]:
predictions

array([-3.32753037e+18, -3.32872151e+18])